In [2]:
import tensorflow as tf

a=tf.random.normal((3,2), 0, 1)

In [3]:
a

<tf.Tensor 'random_normal_1:0' shape=(3, 2) dtype=float32>

In [4]:
type(a)

tensorflow.python.framework.ops.Tensor

In [5]:
a=tf.random.normal((3,2), mean=0, stddev=1)

In [6]:
a

<tf.Tensor 'random_normal_2:0' shape=(3, 2) dtype=float32>

In [7]:
a=tf.random.normal((3,2), mean=0, stddev=1, seed=3)

In [8]:
a

<tf.Tensor 'random_normal_3:0' shape=(3, 2) dtype=float32>

In [15]:
tf.constant([[1.,2.,3.], [4.,5.,6.]])

<tf.Tensor 'Const_2:0' shape=(2, 3) dtype=float32>

In [16]:
tf.constant(43)

<tf.Tensor 'Const_3:0' shape=() dtype=int32>

In [17]:
t=tf.constant([[1.,2.,3.], [4.,5.,6.]])

In [18]:
t.shape

TensorShape([Dimension(2), Dimension(3)])

In [19]:
t.dtype

tf.float32

In [20]:
t+1

<tf.Tensor 'add:0' shape=(2, 3) dtype=float32>

In [22]:
t=t+1

In [23]:
t

<tf.Tensor 'add_1:0' shape=(2, 3) dtype=float32>

In [28]:
t==tf.constant([[2.,3.,4.], [5.,6.,7.]])

False

In [29]:
t+10

<tf.Tensor 'add_4:0' shape=(2, 3) dtype=float32>

In [30]:
tf.constant([1, 2, 3, 4, 5, 6]) 

<tf.Tensor 'Const_7:0' shape=(6,) dtype=int32>

In [32]:
import numpy as np
a= np.array([2., 4., 5.])

In [33]:
a

array([2., 4., 5.])

In [38]:
t=tf.constant([[2.,3.,4.], [5.,6.,7.]])

In [39]:
t

<tf.Tensor 'Const_10:0' shape=(2, 3) dtype=float32>

In [41]:
np.array(t)

array(<tf.Tensor 'Const_10:0' shape=(2, 3) dtype=float32>, dtype=object)

In [44]:
tf.constant(2.)+tf.constant(3)

TypeError: Input 'y' of 'Add' Op has type int32 that does not match type float32 of argument 'x'.

In [49]:
v=tf.Variable([[1,2,3],[4,5,6]])

In [50]:
v==tf.Variable([[1,2,3],[4,5,6]])

False

In [46]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32_ref>

In [48]:
v.assign(2*v)

<tf.Tensor 'Assign:0' shape=(2, 3) dtype=int32_ref>

In [60]:
def huber_fn(y_true, y_pred):
    error=y_true-y_pred
    is_error_small= tf.abs(error)<1
    squared_loss=tf.square(error)
    linear_loss=tf.abs(error)
    return tf.where(is_error_small, squared_loss, linear_loss)

In [61]:
from tensorflow import keras
model=keras.models.Sequential([
    keras.layers.Dense(20, activation="relu")
])

In [62]:
model.compile(loss=huber_fn, optimizer="nadam")

In [63]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error=y_true-y_pred
        is_error_small= tf.abs(error)<1
        squared_loss=tf.square(error)
        linear_loss=tf.abs(error)
        return tf.where(is_error_small, squared_loss, linear_loss)
    return huber_fn

In [65]:
model.compile(loss=create_huber(2.0), optimizer="sgd")

In [66]:
model=keras.models.load_model("keras_model.h5", custom_objects={"huber_fn":create_huber(2.0)})

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [69]:
class HuberLoss(keras.losses.Loss):
    def __init__ (self, threshold=1.0, **kwargs):
        self.threshold= threshold
        super().__init__ (**kwargs)
    def call(self, y_ture, y_pred):
        error=y_true-y_pred
        is_error_small= tf.abs(error)<1
        squared_loss=tf.square(error)
        linear_loss=tf.abs(error)
        return tf.where(is_error_small, squared_loss, linear_loss)
    def get_config(self):
        base_config=super().get_config()
        return {**base_config, "threshold":self.threshold}

In [70]:
model.compile(loss=HuberLoss(2.0), optimizer="nadam")

NameError: name 'y_true' is not defined

In [80]:
def my_activation(z):
    return tf.math.log(z)

In [81]:
def my_initializer(shape, dtype=tf.float32):
    return tf.random.normal(shape, stddev=1, dtype=int32)

In [82]:
def my_regularizer(weights):
    return tf.reduce_sum(weights)

In [83]:
def my_positive_weights(weights):
    return tf.where(weights<0, tf.zeros_like(weights), weights)

In [87]:
layer= keras.layers.Dense(20, kernel_initializer=my_initializer, activation=my_activation, kernel_regularizer=my_regularizer,
                         kernel_constraint=my_positive_weights)

In [89]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor=factor
    def call(self, weights):
        return tf.reduce_sum(weights)
    def get_config(self):
        return {"factor":self.factor}

In [91]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)] )

In [92]:
precision=keras.metrics.Precision()

In [95]:
a=precision([1,0,0],[1,1,0])

In [96]:
b=precision([1,1,1],[1,1,1])

In [99]:
a==np.array(2)

False

In [98]:
b==3

False

In [100]:
p.result()

NameError: name 'p' is not defined

In [101]:
class HuberMetric(keras.metrics.Metric):
    def __init__ (self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        self.huber_fn=create_huber(threshold)
        self.total=self.add_weight("total", initializer="zeros")
        self.count=self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric=self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config=super().get_config()
        return {**base_config, "threshold": self.threshold}

In [105]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units=units
        self.activation=keras.activations.get(activation)
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units], initializer="glorot_normal")
        self.bias= self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
    def call(self,X):
        return self.activation(X @ self.kernel +self.bias)
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units":self.units, "activation":keras.activations.serialize(self.activation)}

In [107]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 =X
        return [X1+X2, X1*X2, X1/X2]
    def compute_output_shape(self, batch_input_size):
        b1, b2 = batch_input_size
        return [b1, b1, b1]

In [108]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev=stddev
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X +noise
        else:
            return X
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

In [110]:
class ResidualBloick(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden=[ keras.layers.Dense(n_neurons, activaton="elu", kernel_initializer="he_normal") for _ in range(n_layers)]
    def call (self, inputs):
        Z=inputs
        for layers in self.hidden:
            Z=layer(Z)
        return inputs+Z

In [112]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden=[keras.layers.Dense(30, activation="selu", kernel_initialization="lecun_normal")]
        self.out=keras.layers.Dense(output_dim)
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct=keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)
    def call(self, inputs):
        Z=inputs
        for layer in self.hidden:
            Z=layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05*recon_loss)
        return self.out(Z)

In [113]:
def f(w1, w2):
    return 3*w1 **2 +2*w1*w2

In [114]:
w1, w2 = 5,3

In [115]:
eps=1e-6

In [116]:
(f(w1+eps, w2)-f(w1,w2))/eps

36.000003007075065

In [117]:
(f(w1, w2+eps)-f(w1,w2))/eps

10.000000003174137

In [118]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)

In [119]:
z= f(w1, w2)

In [120]:
gradient =tf.GradientTape().gradient(z, [w1, w2])

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [121]:
gradient =tf.GradientTape(persistent=True).gradient(z, [w1, w2])

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [130]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z=f(w1, w2)

gradients = tape.gradient(z, [w1, w2])

AttributeError: 'RefVariable' object has no attribute '_id'

In [132]:
with tf.GradientTape() as tape:
    z=f(w1, w2)
dz_dw1= tape.gradient(z, w1)
dz_dw2= tape.gradient(z, w2)
    

AttributeError: 'RefVariable' object has no attribute '_id'

In [134]:
with tf.GradientTape(persistent=True) as tape:
    z= f(w1, w2)
dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(Z, w2)
del tape

AttributeError: 'RefVariable' object has no attribute '_id'

In [135]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z= f(c1, c2)
gradients = tape.gradient(z, [c1, c2])

In [136]:
gradients

[None, None]

In [137]:
def f(w1, w2):
    return 3*w1 **2 +2*w1*w2

In [138]:
z= f(2,3)

In [139]:
z

24

In [140]:
x1, x2 = tf.Variable(5.), tf.Variable(3.)

In [141]:
x1

<tf.Variable 'Variable_17:0' shape=() dtype=float32_ref>

In [142]:
x2

<tf.Variable 'Variable_18:0' shape=() dtype=float32_ref>

In [143]:
c1

<tf.Tensor 'Const_17:0' shape=() dtype=float32>

In [144]:
c2

<tf.Tensor 'Const_18:0' shape=() dtype=float32>

In [145]:
with tf.GradientTape() as tape:
    z= f(x1, x2)
gradients = tape.gradient(z, [x1, x2])

AttributeError: 'RefVariable' object has no attribute '_id'

In [146]:
def f(w1, w2):
    return 3*w1**2 +tf.stop_gradient(2*w1*w2)

In [147]:
with tf.GradientTape() as tape:
    z- f(w1, w2)

In [148]:
gradients = tape.gradient(z, [w1, w2])

AttributeError: 'RefVariable' object has no attribute '_id'

In [149]:
x= tf.Variable([100.])

In [150]:
with tf.GradientTape() as tape:
    z=my_activation(x)

In [151]:
tape.gradient(z, [x])

AttributeError: 'RefVariable' object has no attribute '_id'

In [152]:

def my_better_softplus (z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1+1/exp)
    return tf.math.log(exp+1). my_softplus_graidents

In [153]:
l2_reg = keras.regularizers.l2(0.05)
model= keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [154]:
def random_batch(X, y, batch_size=32):
    idx=np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [155]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics="-".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss]+(metrics or [])])
    end="" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteraton, total)+metrics, end=end)
    

In [157]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing=fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)


n_epochs=5
batch_size = 32
n_steps = len(X_train)//batch_size
optimizer = keras.optimizers.Nadam(lr = 0.01)
loss_fn = keras.losses.mean_squared_error

In [158]:
mean_loss = keras.metrics.Mean()

In [159]:
metrics=[keras.metrics.MeanAbsoluteError]

In [160]:
mean_loss

In [161]:
metrics

[tensorflow.python.keras.metrics.MeanAbsoluteError]

In [165]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
for epoch in range(1, n_epochs +1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps+1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training =True)
            main_loss= tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar (step*batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in  [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [166]:
for variable in model.variables:
    if variable.constraint is not None:
        variable.assign(variable.constraint(variable))

In [169]:
def cube(x):
    return x**3

In [170]:
cube(2)

8

In [171]:
cube(tf.constant(2.0))

<tf.Tensor 'pow_16:0' shape=() dtype=float32>

In [172]:
tf_cube = tf.function (cube)

In [173]:
tf_cube

In [174]:
tf_cube(2)

<tf.Tensor 'PartitionedCall:0' shape=() dtype=int32>

In [175]:
tf_cube(tf.constant(2.0))

<tf.Tensor 'PartitionedCall_1:0' shape=() dtype=float32>

In [176]:
def tf_cube(x):
    return x**3

In [177]:
tf_cube.python_function(3)

AttributeError: 'function' object has no attribute 'python_function'